# Financial Analyst 

### OPEN AI PROJECT-1


In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [2]:
client : OpenAI = OpenAI()


# FETCHING THE API KEY

In [ ]:
key = os.environ["FMP_API_KEY"]
print(key)

# Define financial statement functions
def income_statement(period):     #I GIVE PERIOD THERE 
    url = f"https://financialmodelingprep.com/api/v3/income-statement/AAPL?period=annual&apikey={key}"
    response = requests.get(url)
    return json.dumps(response.json())

print(income_statement())

# CREATE AN ASSISTANT

In [ ]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  instructions="You are a financial analyst",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "income_statement",
      "description": "Give the income statement of quarter and annual from giving the parameter and query",
      "parameters": {
        "type": "object",
        "properties": {
          "period": {"type": "string", "description": "Give the period Annual Or Quarter etc."},
        },
        "required": ["period"]
      }
    }
  }
  ]


# CREATE A THREAD

In [ ]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

# ADD A MESSAGE TO THE THREAD

In [ ]:
from openai.types.beta.threads.thread_message import ThreadMessage

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the Gross Profit in 2019-09-28"
)

dict(message)

# RUN THE ASSISTANT

In [ ]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# MAKE AN VARIABLE TO STORE THE FUNCTION

In [ ]:
available_functions={
    "income_statement":income_statement
}

# POLLING AN UPDATE AND CALLING FOR FUNCTION

In [ ]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "income_statement":
                        
                        response = function_to_call(
                            period=function_args.get("period"),
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break
